In [15]:
import numpy as np
import pickle
import math

In [12]:
SAMPLE_PATH = '/home/catalinlup/MyWorkspace/MasterThesis/datasets/npy_indices/tct_colbert_sample_large.npy'
SAMPLE_PATH_INDICES = '/home/catalinlup/MyWorkspace/MasterThesis/datasets/npy_indices/tct_colbert_sample_large_indices.npy'
PQ_INDEX_PATH = '/home/catalinlup/MyWorkspace/MasterThesis/datasets/quantized_indices/tct_m_8_k_256_200000.pickle'
M = 8
K = 256

In [11]:
original_vectors = np.load(SAMPLE_PATH)
vector_indices = np.load(SAMPLE_PATH_INDICES)
print(original_vectors)

[[ 0.04210953  0.25388956  0.11395846 ... -0.0102931   0.18467581
  -0.1058853 ]
 [ 0.04620727  0.24864034  0.02302429 ... -0.02421323  0.11068066
  -0.14104907]
 [ 0.03764689  0.161587    0.02305674 ... -0.02704101  0.14584206
  -0.11022508]
 ...
 [-0.035588    0.09653375  0.28182703 ... -0.1333796   0.16759278
  -0.05306303]
 [ 0.02289871  0.38744184  0.12625207 ... -0.13288721 -0.13947684
  -0.09221743]
 [ 0.13320503  0.2494825   0.06553803 ...  0.02482365  0.06466706
  -0.10692104]]


In [21]:
pq_index = pickle.load(open(PQ_INDEX_PATH, 'rb'))

def reconstruct_vector(vector_index):
    codebook = pq_index['codebook']
    quantized_index = pq_index['quantized_index']
 
    bits_per_vector = int(math.ceil(math.log2(K)))

    offset_ending = 8 * quantized_index.shape[1] - bits_per_vector * M


    bytes = quantized_index[vector_index, :]
    bit_strings = ['{0:08b}'.format(x)[::-1] for x in bytes]
    bit_string = ''.join(bit_strings)

    bit_string_with_offsets = bit_string[: len(bit_string) - offset_ending]

    sub_vecs = []

    for i in range(M):
        bit_string_for_index = bit_string_with_offsets[i * bits_per_vector : (i + 1) * bits_per_vector]
        index = int(bit_string_for_index[::-1], 2)
        sub_vecs.append(codebook[i, index, :])

    return np.concatenate(sub_vecs)

quantized_vectors = np.vstack([reconstruct_vector(id) for id in vector_indices])

In [23]:
quantized_vectors.shape

(200000, 768)

In [24]:
mean_distortion = np.mean((quantized_vectors - original_vectors) ** 2)

In [25]:
print(mean_distortion)

0.008572319
